<a href="https://colab.research.google.com/github/Ektagavas/CVSummerSchool2021/blob/main/Biometrics/01_FaceRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Face Recognition

In this tutorial, we are going to use the pre-trained VGG-Face network [1] to recognize faces of celebrities. The VGG-Face is a deep-CNN which has been trained using Softmax loss to recognize faces of 2,622 celebrity identities. We will use face images (that have been downloaded from the internet) of a random subset of 10  celebrities, out of the 2,622.

In [ ]:
import math
import cv2
import numpy as np
import glob
import torch
import os
import torchvision.models
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline 
plt.ion()

In [ ]:
# download all the data for this notebook.
# Warning! This data is hosted on a personal Google drive and may be taken off at any time. 
#Also, the file is quite big, so please have patience!

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=16O8OmhRn76m8SlIjPNhnwVnWk2djKhak' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=16O8OmhRn76m8SlIjPNhnwVnWk2djKhak" -O data.zip && rm -rf /tmp/cookies.txt
!unzip data.zip

In [ ]:
! ls

In [ ]:
root = 'data'
eroot = os.path.join(root, 'Experiment_1')
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
def getNameList(filePath):
    names = []
    with open(filePath) as f:
        names = [ line.strip() for line in f ]
    return names

idNames = getNameList(os.path.join(root, "names.txt"))
print("Number of identities = ", len(idNames))
print ("List of the first 5 identities = ", idNames[:5])

Now, let us take a quick look at some of the face images in our dataset. For the convenience of the participants, we have packaged the image names of all the 10 face images in the form of a .t7 data file that can be loaded onto our PyTorch scripts.

In [ ]:
dataset = glob.glob(os.path.join(eroot, '*.*'))
for ii in dataset:
	print(ii)

Let us take the first image in our dataset and visualize what it looks like. Feel free to modify the code if you wish to look at more face images from the dataset!

In [ ]:
dispImg = mpimg.imread(dataset[0])
imgplot = plt.imshow(dispImg)

Before sending the images through the network, we have to perform certain pre-processing steps to make our input image "suitable" for the network. Each CNN has a fixed size of input images that it can accept. The VGG-Face netowrk architecture has been trained on 224 $\times$ 224 RGB images. Therefore, any image that we send as an input must be of 224 $\times$ 224 size.

So, can we simply perform a resize operation on our input image and convert it to the appropriate size? Performing a simple resize operation on arbitrary sized input images may result in a change in the aspect ratio, thereby making the face image look distorted. To avoid that, we perform the following sequence of operations -- 

1. Resize the image such that the smaller dimension (out of height and width) is 256 and the aspect ratio remains the same.
2. Crop a 224 $\times$ 224 region from the center of the resized image.

We will perform the above steps using OpenCV.

In [ ]:
# load the image using OpenCV
inputImg = cv2.imread(dataset[0])

# re-scale the smaller dim (among width, height) to refSize
refSize, targetSize = 256, 224
imgRows, imgCols = inputImg.shape[0], inputImg.shape[1]
if imgCols < imgRows: resizedImg = cv2.resize(inputImg, (refSize, int(refSize * imgRows / imgCols)))
else: resizedImg = cv2.resize(inputImg, (int(refSize * imgCols / imgRows), refSize))
    
# center-crop
oH, oW = targetSize, targetSize
iH, iW = resizedImg.shape[0], resizedImg.shape[1]
anchorH, anchorW = int(math.ceil((iH - oH)/2)), int(math.ceil((iW - oW) / 2))
croppedImg = resizedImg[anchorH:anchorH+oH, anchorW:anchorW+oW]

print(croppedImg.shape)

As you can see, we have resized and cropped our input image to the correct dimensions.

Moving on, the VGG-Face network expects the shape of the input image to be $3 \times 224 \times 224$, which means that the 1st dimension corresponds to the image channel. On the other hand, what we have is an input image whose shape is $224 \times 224 \times 3$ where the last dimension is for the channel. Therefore, in order to make the shapes of the input Tensors consistent, we perform the necessary conversion.

In [ ]:
# convert shape from (height, width, 3) to (3, width, height)
channel_1, channel_2, channel_3 = croppedImg[:, :, 0], croppedImg[:, :, 1], croppedImg[:, :, 2]
croppedImg = np.empty([3, croppedImg.shape[0], croppedImg.shape[1]])
croppedImg[0], croppedImg[1], croppedImg[2] = channel_1, channel_2, channel_3

print(croppedImg.shape)

As a final step, we perform mean subtraction. You might be aware that the input data that is used for training deep CNNs is mean subtracted (this is critical for the stabiliity of the training process). Similarly, while training the VGG-Face network, the per-channel mean of the training images was computed and the same mean value now has to be subtracted for any face image that is used to test the network.

In [ ]:
# subtract training mean
inputImg = inputImg.astype(float)
trainingMean = [129.1863, 104.7624, 93.5940]
for i in range(3): croppedImg[i] = croppedImg[i] - trainingMean[i]

Before we proceed, a quick recap. We have pre-processed our input face images by performing the following steps:
1. Resize (to a slightly bigger size) while maintaining aspect ratio
2. Crop a face patch from the center of the required dimensions
3. Convert to an appropriate shape
4. Subtract the training mean

Moving forward, let us load the pre-trained network and see it's structure.

In [ ]:
def loadVGGModel( filename):
	dat2 = torch.load(filename)
	# copy dictionary
	if str.split(list(dat2.keys())[0],'.')[0] == 'module':
		dat = {}
		for key in dat2.keys():
			k = '.'.join(str.split(key,'.')[1:])
			dat[k] = dat2[key]
	else:
		dat = dat2
		
	n_classes = dat['classifier.6.bias'].shape[0]
	model = torchvision.models.vgg16(pretrained = False)
	lastlayer = torch.nn.Linear(in_features = model.classifier[-1].in_features, \
							   out_features = n_classes, \
							   bias = True)
	model.classifier[-1] = lastlayer
	model.load_state_dict(dat)
	return model

# load pre-trained VGG-Face network
vggFace = loadVGGModel(os.path.join(root, 'VGGFace16.pth'))
vggFace = vggFace.to(device)
print(vggFace)


As you can see, there are 40 layers in total. The architecture can be divided into 5 Convolutional blocks followed by 2 fc layers and a classification layer. Each convolutional block consists of multiple Conv+ReLU layers followed by a pooling layer.

Let us now send our pre-processed input image through the network i.e. we are going to do a forward pass through the pre-trained VGG-Face network.

In [ ]:
# forward pass
img = torch.Tensor(croppedImg).unsqueeze(0)
output = vggFace.forward(img.to(device))
output = output.detach().cpu().numpy()

print(output.shape)

The network has returned a single row of 2,622 entries. These values are the normalized log probabilities of our input face image belonging to each of the 2,622 celebrity IDs in the training set. So, in order to know which ID is the most likely (as per the network), we figure out the one which has the maximum probability.

To get the name of the ID from the index, we use the list of ID names that we had loaded at the start of the tutorial. As you can guess, the order of names in the list is important and serves as a mapping between the index number and the name.

In [ ]:
print("ID = ", idNames[np.argmax(output)])

What are the top-k predictions made by the network for our input image?

In [ ]:
# top-k predictions
ind = output[0].argsort()[-5:][::-1]
for idx in ind:
    print(idNames[idx])

We have seen how to use the pre-trained net to make predictions (face recognition). The VGG-Face net can also be used as a fixed feature extractor for face images. We simply ignore the outputs of the classification layer and take the output of the last fc-layer instead.

In [ ]:
def extractfeatures(x):
	# replicate the forward function
	x = vggFace.features(x)
	x = vggFace.avgpool(x)
	x = torch.flatten(x, 1)
	# forward pass only through the first 4 layers of the classifier
	for ii in range(4):
		x = vggFace.classifier[ii](x)
	return x
	

features = extractfeatures(img.to(device)).detach().cpu().numpy()
print(features.shape)
print(features)


The 4096-d face features that we obtain in this fashion can used for a variety of face-related tasks, as we will see in the subsequent tutorials.

### Exercises

1. Are all the 10 images in the dataset classified correctly?
2. Modify the code to support batch mode of operation -- get predictions (top-1 and top-k) for multiple face images at once.
3. Along with the predicted ID name, also print the probability of prediction.